In [1]:
import pandas
import numpy as np
import sklearn as sk

from sklearn.preprocessing import OneHotEncoder
import time
import pickle

import time
%matplotlib inline
from random import randint
from annoy import AnnoyIndex

In [2]:
dataset_name = "data/raw/data_1.csv"
data = pandas.read_csv(dataset_name).drop(["id1", "id2"], axis=1)
data.head()

,id3,user_id,date
0,714,464300,1
1,581,218912,1
2,254,1072612,1
3,694,967880,1
4,610,557413,1


In [3]:
data = data[data["user_id"].isin(data["user_id"].unique()[:53979])][["user_id", "id3"]].get_values()

In [4]:
users = np.unique(data[:,0])
users_vectors = dict([(user, np.zeros(931)) for user in users])

In [5]:
def ohe(id3):
    vector = np.zeros(931)
    vector[id3] = 1
    
    return vector

In [6]:
for row in data:
    users_vectors[row[0]] += ohe(row[1])

In [7]:
for user in users:
    vec = users_vectors[user]
    users_vectors[user] = vec/vec.sum()

In [10]:
table = []
for key, value in users_vectors.items():
    table.append([key]+value.tolist())

In [28]:
ann = AnnoyIndex(931)

In [29]:
for user in users:
    ann.add_item(user, users_vectors[user])

In [30]:
ann.build(10)

True

In [31]:
items = [i for i in range(931)]

In [33]:
recomendations = []
for uid in list(users_vectors.keys())[:53979]:
    vec = np.mean([users_vectors[i] for i in ann.get_nns_by_item(uid, 70)[1:]],axis=0)
    vec[users_vectors[uid] > 0] = 0
    row = [uid] + [i[1] for i in sorted(list(zip(vec.tolist(), items)), reverse=True)[:5]]
    
    recomendations.append(row)

In [34]:
submit = pandas.DataFrame(data=recomendations, columns=["user_id", "id3_1", "id3_2", "id3_3", "id3_4", "id3_5"])

In [35]:
submit.to_csv("submit.csv", index=False)

In [36]:
submit.head()

,user_id,id3_1,id3_2,id3_3,id3_4,id3_5
0,18,610,800,725,162,700
1,51,581,741,329,310,162
2,73,581,685,875,41,299
3,118,619,662,415,196,145
4,145,204,875,196,586,662
